### Linear regression from scratch using OLS

In [384]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import copy
import seaborn as sns
from sklearn.model_selection import train_test_split

In [385]:
train_data=pd.read_csv('/Users/pushpakumar/Downloads/cs-215-assignment-3-multiple-variate-regression/train.csv')
# test_data=pd.read_csv('/Users/pushpakumar/Downloads/cs-215-assignment-3-multiple-variate-regression/test.csv')

In [386]:
train_corr=train_data.corr()
target = 'yield'
correlation_with_target = train_corr[target]
low_corr_threshold = 0.2
low_corr_features = correlation_with_target[abs(correlation_with_target) < low_corr_threshold].index.tolist()
train_data = train_data.drop(columns=low_corr_features)
x= train_data.drop(columns='yield').values
y = train_data['yield'].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [387]:
# Visualize the data using scatter plot and histogram
# sns.set_palette('colorblind')
# sns.pairplot(data=train_data)

In [388]:
### note input x and y should be panda df

class Multiple_linear_regression():
    def __init__(self):
        self.coefficients=None
        self.intercept=None
        
    def fit(self,x,y):
        x=self.transform_x(x)
        y=self.transform_y(y)
        betas=self.estimate_coefficients(x,y)
        self.intercept=betas[0]
        self.coefficients=betas[1:]

            
    def predict(self, x):
        ## y = b_0 + b_1*x + ... + b_i*x_i
        prediction = np.dot(x, self.coefficients) + self.intercept
        return prediction
    
    def r2_score(self, y_true, y_pred):
        '''
            r2 = 1 - (rss/tss)
            rss = sum_{i=0}^{n} (y_i - y_hat)^2
            tss = sum_{i=0}^{n} (y_i - y_bar)^2
        '''
        y_values=y_true
        y_avg=np.mean(y_values)
        
        residual_sum_of_squares=0
        total_sum_of_squares=0
        for i in range(len(y_values)):
            residual_sum_of_squares+=(y_values[i]-y_pred[i])**2
            total_sum_of_squares+=(y_values[i]-y_avg)**2
            
        return 1-(residual_sum_of_squares/total_sum_of_squares)
    
    def transform_x(self, x):
        return np.hstack((np.ones((x.shape[0], 1)), x))
    def transform_y(self,y):
        return y
    def estimate_coefficients(self,x,y):
        # (x'x)^-1 x'y 
        xT=x.transpose()
        inversed=np.linalg.inv(xT.dot(x))
        coefficients=inversed.dot(xT).dot(y) 
        return coefficients

In [389]:
class Nadaraya_watson_kernel_guassian():
    def __init__(self,bandwidth=1.0):
        self.bandwidth=bandwidth
        self.x_data=None
        self.y_data=None
    def fit(self,x,y):
        self.x_data=np.array(x)
        self.y_data=np.array(y)
    def gaussian_kernel(self,x):
        ## (1/2*pi) e^{-x^2 / 2}
        exponent=np.exp(-0.5 * (x**2))
        result=exponent/np.sqrt(2*np.pi)
        return result
    
    def predict(self, x):
        # n,m=x.shape
        temp = (x - self.x_data) / self.bandwidth
        distance_from_origin=np.linalg.norm(temp,axis=1)
        ker = self.gaussian_kernel(distance_from_origin)
        num = np.sum(ker * self.y_data)
        den = np.sum(ker)
        return num / den           
    
    def risk_function(self, x, y):
        # x is prediction y is actual 
        return (y - x)**2

In [390]:
def mean_absolute_error(x,y):
    n=len(x)
    error=0
    for i in range(n):
        error+=abs(x[i]-y[i])
    return error/n

In [391]:
mlr=Multiple_linear_regression()
mlr.fit(x_train,y_train)
prediction_mlr=mlr.predict(x_train)
score=mlr.r2_score(y_train,prediction_mlr)
print(score)

0.9039003705337196


In [392]:
print(mean_absolute_error(prediction_mlr,y_train))


273.6036825812598


In [393]:
# print(h)

In [394]:
# x_train_h, x_test_h, y_train_h, y_test_h = train_test_split(x_train, y_train, test_size=0.1, random_state=0)

In [395]:
# h_x=np.linspace(0,3,200)
# risk=[]
# for i in h_x:
#     mlr2=Nadaraya_watson_kernel_guassian(i)
#     mlr2.fit(x_train_h,y_train_h)
#     risk_temp=0
#     for j in range(len(x_test_h)):
#         temp=mlr2.predict(x_test_h[j])
#         risk_temp+=mlr2.risk_function(temp,y_test_h[j])
#     risk.append(risk_temp/len(x_train_h))
# plt.plot(h_x,risk)

In [396]:
# risk=np.array(risk)
# risk_witout_nan=risk[~np.isnan(risk)]
# print(risk_witout_nan.min())
# index = np.where(risk== risk_witout_nan.min())[0][0]
# print(index)
# h=h_x[index]
# print(h)

In [397]:
# kr=Nadaraya_watson_kernel_guassian(h)
# kr.fit(x_train_h,y_train_h)
# prediction_kr_h=[]
# for i in range(len(x_test_h)):
#     temp=kr.predict(x_test_h[i])
#     prediction_kr_h.append(temp)
    
# print(prediction_kr_h[0:15])

# print(y_train_h[0:15])

In [398]:
# plt.scatter(y_test_h,prediction_kr_h)
# plt.plot(y_test,y_test,color='red')
# plt.legend()

In [399]:
# np.corrcoef(y_test_h,prediction_kr_h)

In [400]:
kr=Nadaraya_watson_kernel_guassian(0.3015)
kr.fit(x_train,y_train)
prediction_kr=[]
for i in range(len(x_train)):
    temp=kr.predict(x_train[i])
    prediction_kr.append(temp)
    
print(prediction_kr[0:15])

print(y_train[0:15])

[6281.3504758268255, 6166.060967587263, 4437.9792864478795, 3981.2659748168808, 5339.801637289458, 8176.205745072179, 5533.258636582647, 6388.119713837324, 7312.072359168474, 4452.053807260963, 6335.062014292895, 6006.74833688852, 5400.626323583488, 6893.528182097134, 5408.006470104926]
[8969.40184 5966.71128 3385.148   4489.22384 5637.9843  8157.23694
 5750.88107 5747.56017 7122.49056 4045.47966 6754.44894 6251.61184
 5063.85109 7041.38018 5920.48718]


In [401]:
print(mean_absolute_error(y_train,prediction_kr))

326.3696240570526


In [402]:
prediction_mlr=np.array(prediction_mlr)
prediction_kr=np.array(prediction_kr)
x_train_final = np.column_stack((prediction_mlr, prediction_kr))
y_train_final=y_train
print(x_train_final[0])
print((x_train_final.shape))
print()

[6416.93815453 6281.35047583]
(12000, 2)



In [403]:
weight_linear=1/273.6036825812598
weight_kernel=1/326.2866463929124
total=weight_linear+weight_kernel
weight_linear/=total
weight_kernel/=total
yy=(weight_linear*prediction_mlr)+(weight_kernel*prediction_kr)

In [404]:
print(mean_absolute_error(y_train_final,yy))

283.0236764706015


In [416]:
prediction_mlr_test=mlr.predict(x_test)
prediction_kr_test=[]
for i in range(len(x_test)):
    temp=kr.predict(x_test[i])
    prediction_kr_test.append(temp)
prediction_mlr_test=np.array(prediction_mlr_test)
prediction_kr_test=np.array(prediction_kr_test)
x_train_final = np.column_stack((prediction_mlr, prediction_kr))
y_train_final=y_test
print(x_train_final[0])
print((x_train_final.shape))
print()

[6416.93815453 6281.35047583]
(12000, 2)



In [419]:
weight_linear=1/273.6036825812598
weight_kernel=1/326.2866463929124
total=weight_linear+weight_kernel
weight_linear/=total
weight_kernel/=total
yy=(weight_linear*prediction_mlr_test)+(weight_kernel*prediction_kr_test)

In [420]:
print(mean_absolute_error(y_train_final,yy))

276.3110084825565


In [405]:
# x_train_final_h, x_test_final_h, y_train_final_h, y_test_final_h = train_test_split(x_train_final, y_train_final, test_size=0.1, random_state=0)

In [406]:
# print(y_train_final_h[0:5])
# print(x_train_final_h[0:5])

In [407]:
# h_x=np.linspace(0,200,200)
# risk=[]
# for i in h_x:
#     mlr2=Nadaraya_watson_kernel_guassian(i)
#     mlr2.fit(x_train_final_h,y_train_final_h)
#     risk_temp=0
#     for j in range(len(x_test_final_h)):
#         temp=mlr2.predict(x_test_final_h[j])
#         risk_temp+=mlr2.risk_function(temp,y_test_final_h[j])
#     risk.append(risk_temp/len(x_train_final_h))
# plt.plot(h_x,risk)


In [408]:
# print(risk)

In [409]:
# risk=np.array(risk)
# risk_witout_nan=risk[~np.isnan(risk)]
# print(risk_witout_nan.min())
# index = np.where(risk== risk_witout_nan.min())[0][0]
# print(index)
# h=h_x[index]
# print(h)

In [410]:
# mlr2=Multiple_linear_regression()
# mlr2.fit(x_train_final,y_train_final)
# prediction_mlr2=[]
# for i in x_train_final:
#     temp=mlr2.predict(i)
#     prediction_mlr2.append(temp)
# # score=mlr.r2_score(y_test,prediction_mlr)
# print(prediction_mlr2[0:5])
# print(y_train_final[0:5])

In [411]:
# plt.scatter(prediction_mlr2,y_train_final)

In [412]:
# np.corrcoef(prediction_mlr2,y_train_final)

In [413]:
# print(mean_absolute_error(y_train_final_h,prediction_mlr2))